## Scraping The Numbers: https://www.the-numbers.com/movie/budgets/all

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import duckdb
import pandas as pd
import re

from ydata_profiling import ProfileReport
import warnings

import functions_project

warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Scrape first page
driver = webdriver.Chrome()
driver.get("https://www.the-numbers.com/movie/budgets/all")
html = driver.page_source
df = pd.read_html(html)[0]
next_urls = driver.find_element(By.CLASS_NAME, "pagination")
next_page = re.findall("(.movie.+)\"", next_urls.get_attribute("outerHTML"))[1]

# Scrape all other pages
end_of_scraping = False

while end_of_scraping == False:
    driver.get("https://www.the-numbers.com" + next_page)
    html = driver.page_source
    budget = pd.read_html(html)[0]
    df = df.append(budget)
    driver.find_element(By.CLASS_NAME, "pagination")
    next_urls = driver.find_element(By.CLASS_NAME, "pagination")
    s = next_urls.get_attribute("outerHTML")
    s = s[s.find('active'):]
    try:
        next_page = re.findall("(.movie.+)\"", s)[0]        
    except:
        end_of_scraping = True

# Close scraper        
driver.close()

In [10]:
# Data wrangling dataset
budget_df = df[df['Release Date'] != 'Unknown']
budget_df = budget_df.drop(['Unnamed: 0'], axis=1)
budget_df['Release Date'] = pd.to_datetime(budget_df['Release Date'])
budget_df['Release Year'] = budget_df['Release Date'].dt.year
functions_project.map_cleaned_strings(budget_df, 'Movie')
budget_df = budget_df.rename(columns={"Release Date": "release_date", "Movie": "movie", "Production Budget": "production_budget", 
                          "Domestic Gross": "domestic_gross", "Worldwide Gross": "worldwide_gross", "Release Year": "release_year"})

budget_df

release_date                                       movie production_budget  \
0    2022-12-09                     avatar the way of water      $460,000,000   
1    2019-04-23                            avengers endgame      $400,000,000   
2    2011-05-20  pirates of the caribbean on stranger tides      $379,000,000   
3    2015-04-22                      avengers age of ultron      $365,000,000   
4    2023-05-17                                      fast x      $340,000,000   
..          ...                                         ...               ...   
72   1999-04-02                                   following            $6,000   
73   2005-07-13               return to the land of wonders            $5,000   
74   2015-09-29                        a plague so pleasant            $1,400   
75   2005-08-05                           my date with drew            $1,100   
76   2021-03-02                                      neeras               $86   

   domestic_gross worldwide_gross  release_year  
0    $678,465,888  $2,303,465,888          2022  
1    $858,373,000  $2,794,731,755          2019  
2    $241,071,802  $1,045,713,802          2011  
3    $459,005,868  $1,395,316,979          2015  
4              $0              $0          2023  
..            ...             ...           ...  
72        $48,482        $240,495          1999  
73         $1,338          $1,338          2005  
74             $0              $0          2015  
75       $181,041        $181,041          2005  
76             $0              $0          2021  

[6264 rows x 6 columns]

In [6]:
# Perform EDA report on budget data set
profile = ProfileReport(budget_df, title="Report")
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
# create a connection to a file called 'imdb_reviews.db'
con = duckdb.connect('imdb_reviews.db')

# create a table and load data into it
con.sql("DROP TABLE IF EXISTS budget_movie_data")
con.sql("CREATE TABLE IF NOT EXISTS budget_movie_data AS SELECT * FROM budget_df")

con.close()